# Basic DTW Code
Code from https://towardsdatascience.com/dynamic-time-warping-3933f25fcdd  
This code implements a simple DTW measure between two series represented as lists.  
It allows for series of different lengths and has a `window` parameter that determines the amount of warping allowed.  
For series of different lengths, the minimum warping will be the difference in the lengths.  

In [75]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [76]:
def dtw(s, t, window):
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    #initialize cells (1...n, 0) and cells (0, 1...m) as infinite, cell (0,0) as 0
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    #calculate the value of the rest cells in the n*m size table
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0

    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    # print(dtw_matrix)
    return dtw_matrix[-1, -1]

In [77]:
a = [1,2,3,3,5]
b = [1,2,2,2,2,2,2,4]
dtw(a,b, window = 3)

3.0

In [78]:
x1 = [7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7]
x2 = [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7]

In [79]:
dtw_0 = dtw(x1,x2,window = 1)
dtw_0

18.0

Works also for numpy arrays.

In [80]:
x = np.array([[7,7,8,9,10,10,7,4,2,1,2,4,7,11,10,9,7],
                [7,8,10,10,8,7,3,2,2,4,6,12,12,9,7,7,8]])

In [81]:
x[0]

array([ 7,  7,  8,  9, 10, 10,  7,  4,  2,  1,  2,  4,  7, 11, 10,  9,  7])

In [82]:
for w in range(10):
    dtw_n = dtw(x[0],x[1],window = w)
    print(w, dtw_n)

0 43.0
1 19.0
2 9.0
3 9.0
4 9.0
5 9.0
6 9.0
7 9.0
8 9.0
9 9.0


# Task 1: 1-NN DTW

## 1. Use the DTW function in the notebook to implement a simple 1-NN classifier for time-series data.
Here I defined three methods to implement 1-NN classfier: calc_distance_dtw(), get_neighbor(), and get prediction()

In [83]:
def calc_distance_dtw(s, t, window):
    """Function use DTW algo to calculate distance between two series"""
    n, m = len(s), len(t)
    w = np.max([window, abs(n-m)]) # warping cannot be less than the difference in lengths. 
    dtw_matrix = np.zeros((n+1, m+1))
    
    #initialize cells (1...n, 0) and cells (0, 1...m) as infinite, cell (0,0) as 0
    for i in range(n+1):
        for j in range(m+1):
            dtw_matrix[i, j] = np.inf
    dtw_matrix[0, 0] = 0
    
    #calculate the value of the rest cells in the n*m size table
    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            dtw_matrix[i, j] = 0

    for i in range(1, n+1):
        for j in range(np.max([1, i-w]), np.min([m, i+w])+1):
            cost = abs(s[i-1] - t[j-1])
            # take last min from a square box
            last_min = np.min([dtw_matrix[i-1, j], dtw_matrix[i, j-1], dtw_matrix[i-1, j-1]])
            dtw_matrix[i, j] = cost + last_min
    return dtw_matrix[-1, -1]

def get_neighbor(test_row, train_dataset, window):
    """Function takes test_row and dataset (a dataframe) as parameters, returns nearest row in the dataset to the test_row"""
    #a list to store all distances between rows and single test row
    distances = []
    #iterate through dataset, skip test_row
    for index in train_dataset.index:
        distances.append(calc_distance_dtw(s = list(test_row), t=list(train_dataset.iloc[index]), window=window))
    #convert the list to a dataframe, use the index from dataset
    df_distances = pd.DataFrame(data=distances, index=train_dataset.index, columns=['distance'])
    #sort the distances by values, take the smallest(nearest) one, say, the first row
    df_neighbor = df_distances.sort_values(axis=0, by='distance')
    #return this row
    return df_neighbor.iloc[0]

def get_prediction(df_neighbor, y_classes):
    """Function takes the result from get_neighbor() function and y/target dataframe, returns the prediction of test_row"""
    #get the laebl/class of the nearest neighbor, make prediction
    prediction = float(y_classes.iloc[df_neighbor.name])
    return prediction

## 2. Test the performance of this classifier on the dataset provided in the file UMD_TEST.txt. This is a three-class synthetic time-series dataset.

In [84]:
import pandas as pd
from sklearn.model_selection import train_test_split

#read the text file to a dataframe
df = pd.read_table("UMD_TEST.txt", delimiter="  ", header=None)
#derive target column from original dataset, assign it to y_classes
target_dataset = pd.DataFrame(df.iloc[:, 0])
#drop target column in original dataset, assign the rest to train_dataset
feature_dataset = df.drop(0, axis=1)
#spilt the data to 75%(train) and 25%(test), and reset index for them
X_train, X_test, y_train, y_test = train_test_split(feature_dataset, target_dataset, test_size=0.25, random_state=0)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [85]:
def performance_oneNN_DTW(window):
    #list to store all predicted values for X_test
    y_test_pred = []
    result_dict = {}
    #iterate through rows in X_test set, make predictions for each test_row based on X_train set
    for i in range(X_test.shape[0]):
        df_neighbor = get_neighbor(test_row=X_test.iloc[i], train_dataset=X_train, window=window)
        prediction_result = get_prediction(df_neighbor=df_neighbor, y_classes=y_train)
        y_test_pred.append(prediction_result)

    #generate the accuracy score for this classfier
    accuracy = accuracy_score(y_test, y_test_pred)
    #generate confusion matrix for this classfier
    confusion =  confusion_matrix(y_test, y_test_pred)
    #store all results to a dict
    result_dict['window_size'] = window
    result_dict['accuracy_score'] = accuracy
    result_dict['confusion_matrix'] = confusion

    return result_dict

#call the function with window size == 0
performance_oneNN_DTW(window=0)

{'window_size': 0,
 'accuracy_score': 0.8611111111111112,
 'confusion_matrix': array([[13,  2,  0],
        [ 0,  8,  0],
        [ 0,  3, 10]])}

## 3. Given that the time-series are all the same length (150 ticks) Euclidean distance can also be used as distance metric. Compare the 1-NN DTW performance with 1-NN Euclidean. Use the scikit-learn implementation for the Euclidean model.

In [86]:
from sklearn.neighbors import KNeighborsClassifier

#new a class instance, p = 2 means this will be equivalent to Euclidean distance
sklearn_classfier = KNeighborsClassifier(n_neighbors=1, p=2)
#fit the classfier from the train dataset
sklearn_classfier.fit(X_train.to_numpy(), y_train.to_numpy())
#predict test dataset
y_test_pred = sklearn_classfier.predict(X_test.to_numpy())
print("Prediction results are:", y_test_pred)
#accuracy score
print("Accuracy score for sklearn oneNN Classfier is:", accuracy_score(y_test, y_test_pred))
#confusion matrix
print("Confusion matrix of sklearn oneNN Classifer is:\n", confusion_matrix(y_test, y_test_pred))

Prediction results are: [1. 2. 3. 2. 3. 3. 2. 1. 2. 1. 3. 3. 2. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn oneNN Classfier is: 0.8611111111111112
Confusion matrix of sklearn oneNN Classifer is:
 [[12  3  0]
 [ 0  8  0]
 [ 0  2 11]]


<h5><b>Conclusion</b></h5>
<p><b>
In Task1-2, we've got the following results:<br>
&nbsp;&nbsp;{'window_size': 0,<br>
&nbsp;&nbsp;'accuracy_score': 0.8611111111111112,<br>
&nbsp;&nbsp;'confusion_matrix': <br>
    &nbsp;&nbsp;&nbsp;([[13,  2,  0],<br>
    &nbsp;&nbsp;&nbsp;[ 0,  8,  0],<br>
    &nbsp;&nbsp;&nbsp;[ 0,  3, 10]])}<br>
</p></b>

<p><b>
Here, we got:<br>
&nbsp;&nbsp; 'accuracy_score': 0.8611111111111112<br>
&nbsp;&nbsp; 'Confusion Matrix': <br>
&nbsp;&nbsp;&nbsp;[[12  3  0]<br>
&nbsp;&nbsp;&nbsp;[ 0  8  0]<br>
&nbsp;&nbsp;&nbsp;[ 0  2 11]]<br>
</p></b>

<p><b>
Comparing the two, we can see that they both have the same accuracy score, there is a small difference in their confusion matrix, but these two matrices are similar matrices because they have the same rank.<br>
We can therefore conclude that the two are in agreement in terms of accuracy, but there are very small differences in the prediction of certain categories.
</p></b>

## 4. Find the best value for the window parameter for this dataset.

In [87]:
#here we use a for loop to test which window size will get the best accuracy score
for i in range(10):
    print(performance_oneNN_DTW(window=i))

{'window_size': 0, 'accuracy_score': 0.8611111111111112, 'confusion_matrix': array([[13,  2,  0],
       [ 0,  8,  0],
       [ 0,  3, 10]])}
{'window_size': 1, 'accuracy_score': 0.8611111111111112, 'confusion_matrix': array([[13,  2,  0],
       [ 0,  8,  0],
       [ 0,  3, 10]])}
{'window_size': 2, 'accuracy_score': 0.8888888888888888, 'confusion_matrix': array([[13,  2,  0],
       [ 0,  8,  0],
       [ 0,  2, 11]])}
{'window_size': 3, 'accuracy_score': 0.9166666666666666, 'confusion_matrix': array([[13,  2,  0],
       [ 0,  8,  0],
       [ 0,  1, 12]])}
{'window_size': 4, 'accuracy_score': 0.9166666666666666, 'confusion_matrix': array([[13,  2,  0],
       [ 0,  8,  0],
       [ 0,  1, 12]])}
{'window_size': 5, 'accuracy_score': 0.9444444444444444, 'confusion_matrix': array([[14,  1,  0],
       [ 0,  8,  0],
       [ 0,  1, 12]])}
{'window_size': 6, 'accuracy_score': 0.9444444444444444, 'confusion_matrix': array([[14,  1,  0],
       [ 0,  8,  0],
       [ 0,  1, 12]])}
{'wind

<h5><b>Conclusion</b></h5>
<p><b>From the output we can conclude that when window size is greater than or equals to 5, the accuracy score will be the highest, which is 0.944</p></b>

# Task 2: k-NN DTW

## 1. The k-NN classifier in scikit-learn has a metric parameter that allows for user-defined distance metrics. Adapt the DTW code provided so that it can be incorporated in a scikit-learn k-NN classifier.

In [88]:
#set the parameter 'metric' to my own function 'calc_distance_dtw()', so the sklearn classfier will use this function to calculate distance; we still set number of neighbors to 1, window size to 0.
sklearn_classfier = KNeighborsClassifier(metric=calc_distance_dtw, n_neighbors=1, metric_params={'window': 0})

## 2. Test the performance of this classifier and compare with the 1-NN results from Task 1. Verify that the 1-NN results are consistent.

In [89]:
#test it
#fit the classfier with train set
sklearn_classfier.fit(X_train.to_numpy(), y_train.to_numpy())
#make predictions
y_test_pred = sklearn_classfier.predict(X_test.to_numpy())
print("Prediction results are:", y_test_pred)
#check accuracy
print("Accuracy score for sklearn kNN Classfier (DTW distance) is:\n", accuracy_score(y_test, y_test_pred))
#check confusion metrics
print("Confusion matrix of sklearn kNN Classifer (DTW distance) is:\n", confusion_matrix(y_test, y_test_pred))

Prediction results are: [1. 2. 3. 2. 3. 2. 2. 1. 2. 1. 3. 3. 2. 1. 2. 1. 1. 1. 3. 2. 2. 3. 1. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (DTW distance) is:
 0.8611111111111112
Confusion matrix of sklearn kNN Classifer (DTW distance) is:
 [[13  2  0]
 [ 0  8  0]
 [ 0  3 10]]


<h5><b>Conclusion</b></h5>
<p><b>
In Task1-2, we've got the following results:<br>
&nbsp;&nbsp;{'window_size': 0,<br>
&nbsp;&nbsp;'accuracy_score': 0.8611111111111112,<br>
&nbsp;&nbsp;'confusion_matrix': <br>
    &nbsp;&nbsp;&nbsp;([[13,  2,  0],<br>
    &nbsp;&nbsp;&nbsp;[ 0,  8,  0],<br>
    &nbsp;&nbsp;&nbsp;[ 0,  3, 10]])}<br>
</p></b>

<p><b>
Here we got:
&nbsp;&nbsp;'accuracy_score':<br>
0.8611111111111112
&nbsp;&nbsp;'confusion_matrix':
 [[13  2  0]
 [ 0  8  0]
 [ 0  3 10]]
</p></b>

<p><b>By comparing the two, the results are found to be identical.</p></b>

## 3. Compare with k-NN Euclidean.

### 3.1 k-NN Euclidean with different number of neighbors

In [90]:
from datetime import datetime
start = datetime.now()
#don't specify parameter 'metric', set 'p' = 2 instead to apply Euclidean distance, try different number of neighbors, see the difference among results
for i in (1, 3, 5, 7):
    print("Number of neighbors:", i)
    sklearn_classfier = KNeighborsClassifier(n_neighbors=i, p=2)

    #fit the classfier from the train dataset
    sklearn_classfier.fit(X_train.to_numpy(), y_train.to_numpy())
    #predict test dataset
    y_test_pred = sklearn_classfier.predict(X_test.to_numpy())
    print("Prediction results are:\n", y_test_pred)
    #accuracy score
    print("Accuracy score for sklearn kNN Classfier (Euclidean) is:\n", accuracy_score(y_test, y_test_pred))
    #confusion matrix
    print("Confusion matrix of sklearn kNN Classifer (Euclidean) is:\n", confusion_matrix(y_test, y_test_pred))
    #divider
    print("**********DIVIDER**********\n")
end = datetime.now()
print("Time spend for sklearn kNN Classfier (Euclidean):", end-start)

Number of neighbors: 1
Prediction results are:
 [1. 2. 3. 2. 3. 3. 2. 1. 2. 1. 3. 3. 2. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (Euclidean) is:
 0.8611111111111112
Confusion matrix of sklearn kNN Classifer (Euclidean) is:
 [[12  3  0]
 [ 0  8  0]
 [ 0  2 11]]
**********DIVIDER**********

Number of neighbors: 3
Prediction results are:
 [1. 2. 3. 2. 3. 2. 2. 1. 2. 1. 3. 3. 2. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (Euclidean) is:
 0.8333333333333334
Confusion matrix of sklearn kNN Classifer (Euclidean) is:
 [[12  3  0]
 [ 0  8  0]
 [ 0  3 10]]
**********DIVIDER**********

Number of neighbors: 5
Prediction results are:
 [1. 2. 3. 2. 3. 2. 2. 1. 2. 1. 3. 2. 2. 2. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (Euclidean) is:
 0.7777777777777778
Confusion matrix of sklearn kNN Classifer 

### 3.2 k-NN DTW with different number of neighbors

In [91]:
#try different number of neighbors, see the difference among results, use the best window size found from above task
start = datetime.now()
for i in (1,3,5,7):
    print("Number of neighbors:", i)
    sklearn_classfier = KNeighborsClassifier(metric=calc_distance_dtw, n_neighbors=i, metric_params={'window': 5})
    #fit the classfier with train set
    sklearn_classfier.fit(X_train.to_numpy(), y_train.to_numpy())
    #make predictions
    y_test_pred = sklearn_classfier.predict(X_test.to_numpy())
    print("Prediction results are:", y_test_pred)
    #check accuracy
    print("Accuracy score for sklearn kNN Classfier (DTW) is:\n", accuracy_score(y_test, y_test_pred))
    #check confusion metrics
    print("Confusion matrix of sklearn kNN Classifer (DTW) is:\n", confusion_matrix(y_test, y_test_pred))
    #divider
    print("**********DIVIDER**********\n")
end = datetime.now()
print("Time spend for sklearn kNN Classfier (Euclidean):", end-start)

Number of neighbors: 1
Prediction results are: [1. 2. 3. 1. 3. 3. 2. 1. 3. 1. 3. 3. 1. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (DTW) is:
 0.9444444444444444
Confusion matrix of sklearn kNN Classifer (DTW) is:
 [[14  1  0]
 [ 0  8  0]
 [ 0  1 12]]
**********DIVIDER**********

Number of neighbors: 3
Prediction results are: [1. 2. 3. 1. 3. 2. 2. 1. 2. 1. 3. 3. 1. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (DTW) is:
 0.8888888888888888
Confusion matrix of sklearn kNN Classifer (DTW) is:
 [[14  1  0]
 [ 0  8  0]
 [ 0  3 10]]
**********DIVIDER**********

Number of neighbors: 5
Prediction results are: [1. 2. 3. 1. 3. 2. 2. 1. 2. 1. 2. 2. 1. 1. 2. 1. 1. 1. 3. 2. 2. 3. 2. 3.
 1. 3. 2. 2. 3. 1. 2. 1. 3. 1. 2. 1.]
Accuracy score for sklearn kNN Classfier (DTW) is:
 0.8333333333333334
Confusion matrix of sklearn kNN Classifer (DTW) is:
 [[14  1  0]
 [ 0  8  0

<h5><b>Conclusion</b></h5>
<p><b>Compare the results of Euclidean Classfier and DTW Classfier, we can draw conclusions in the following areas:</b></p>
<p><b>
&nbsp;&nbsp;- Accuracy: When we assign the best window size (which is 5 I found) to the DTW Classfier, the accuracy scores are always higher than those from the Euclidean Classfier when tesing under different numbers of neighbor. So, when we want to reach higher accuracy, DTW may be the choice.
</b></p>
<p><b>
&nbsp;&nbsp;- Efficiency: However, when it comes to efficiency, the Euclidean Classfier consumes much less time than the DTW Classfier when the number of neighbor is relatively greater. So when there is not enough computational resource or efficiency is prefered, the Euclidean Classfier may be prefered.
</b></p>
<p><b>
Besides, for this dataset, the greater the number of neighbor is, the lower accuracy score will be.
</b></p>